# Introduction

This notebook provides a comprehensive approach to **Output Moderation** for text generated by language models or other AI systems. As AI applications increasingly interact with users, ensuring the quality, relevance, and safety of the generated outputs is paramount. 

The provided code implements key moderation techniques to address critical concerns such as:

1. **Hallucination Prevention**: Ensures that generated outputs align with the given context and avoid fabricated or misleading content.
2. **Sanitization**: Treats all outputs as potentially harmful, escaping any special characters to mitigate injection attacks.
3. **Toxicity Filtering**: Uses a predefined set of toxic words and regular expressions to detect and prevent harmful language.
4. **Anomaly Detection**: Identifies outputs with abnormal patterns, such as excessive repetition or nonsensical content.
5. **Monitoring and Logging**: Tracks outputs along with contextual metadata to enable thorough analysis and debugging.
6. **Feedback Mechanism**: Provides a mechanism for user feedback to refine and improve moderation strategies over time.

## Objective

The goal of this notebook is to act as a **baseline framework** for implementing and enhancing moderation pipelines in AI systems. This solution can be used as a starting point for developers aiming to ensure that their models generate outputs that are:
- **Safe** (free from toxic or harmful content),
- **Relevant** (contextually appropriate), and
- **Trustworthy** (minimally prone to hallucinations or anomalies).

## Use Cases

This framework is particularly suited for:
- Chatbots or conversational AI systems.
- Generative AI models producing text summaries or insights.
- Applications in sensitive domains such as healthcare, education, or customer support, where content moderation is critical.

By following the outlined moderation steps, developers can safeguard their applications, enhance user trust, and align with ethical AI practices.


In [6]:
intents = 'hello.yes, hi.no'

if len(intents) > 0:
    intents_list = [f"'{intent.strip()}'" for intent in intents.split(",")]
    intent_query = "AND ul.intent_name IN ({0})".format(", ".join(intents_list))
intent_query

"AND ul.intent_name IN ('hello.yes', 'hi.no')"

In [1]:
import re
import html
from datetime import datetime

# Configuration for Moderation
TOXIC_WORDS = {"hate", "violence", "malicious", "abuse", "harm"}
ANOMALY_REPETITION_THRESHOLD = 3  # Number of unique words required to avoid flagging as repetitive
MAX_OUTPUT_LENGTH = 500  # Maximum allowed length for output

# Functions for Output Moderation
def hallucination_prevention(output: str, context: str) -> bool:
    """
    Check if the output aligns with the provided context to prevent hallucination.
    """
    return context.lower() in output.lower()

def treat_as_untrusted(output: str) -> str:
    """
    Treat output as untrusted and sanitize it for safe use.
    """
    return html.escape(output)

def filter_toxicity(output: str) -> bool:
    """
    Detect and filter toxic content in the output using predefined toxic words.
    Uses regex to identify toxic patterns.
    """
    for toxic_word in TOXIC_WORDS:
        if re.search(rf"\b{toxic_word}\b", output, re.IGNORECASE):
            return False
    return True

def anomaly_detection(output: str) -> bool:
    """
    Detect anomalies in output, such as excessive repetition, nonsensical content, or length violations.
    """
    words = output.split()
    unique_words = set(words)
    
    # Check for excessive repetition
    if len(unique_words) < ANOMALY_REPETITION_THRESHOLD:
        return False
    
    # Check for excessive length
    if len(output) > MAX_OUTPUT_LENGTH:
        return False
    
    # Additional checks can be added as needed (e.g., nonsensical patterns)
    return True

def monitor_and_log(output: str, context: str, status: str):
    """
    Monitor output and log for analysis and debugging, including timestamps and context.
    """
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open("output_logs.txt", "a") as log_file:
        log_file.write(
            f"[{timestamp}] Context: {context}\n"
            f"Output: {output}\n"
            f"Status: {status}\n\n"
        )

def feedback_mechanism(output: str, is_acceptable: bool):
    """
    Allow user feedback to refine future moderation decisions.
    """
    feedback_file = "moderation_feedback.txt"
    with open(feedback_file, "a") as file:
        feedback = "Acceptable" if is_acceptable else "Unacceptable"
        file.write(f"Output: {output}\nFeedback: {feedback}\n\n")

# Main Execution with Example Usage
if __name__ == "__main__":
    # Example context and outputs
    context = "Explain the concept of artificial intelligence."
    outputs = [
        "Artificial intelligence is the simulation of human intelligence in machines.",
        "I hate the concept of AI, it's promoting violence.",
        "Hello Hello Hello",
        "<script>alert('malicious');</script>",
        "Artificial intelligence is a field that <b>abuses</b> resources."
    ]

    for output in outputs:
        print(f"Original Output: {output}")

        # Treat output as untrusted and sanitize it
        safe_output = treat_as_untrusted(output)
        print(f"Sanitized Output: {safe_output}")

        # Check for hallucinations
        if not hallucination_prevention(safe_output, context):
            status = "Hallucinated or irrelevant content detected."
            print(f"Warning: {status}")
            monitor_and_log(safe_output, context, status)
            continue

        # Filter for toxicity
        if not filter_toxicity(safe_output):
            status = "Toxic or harmful content detected."
            print(f"Warning: {status}")
            monitor_and_log(safe_output, context, status)
            feedback_mechanism(safe_output, False)
            continue

        # Detect anomalies
        if not anomaly_detection(safe_output):
            status = "Anomaly detected in content (e.g., repetition, length)."
            print(f"Warning: {status}")
            monitor_and_log(safe_output, context, status)
            feedback_mechanism(safe_output, False)
            continue

        # Log output for monitoring
        status = "Output accepted."
        monitor_and_log(safe_output, context, status)
        print("Output logged successfully.\n")

        # Collect user feedback (mock example for demonstration)
        feedback_mechanism(safe_output, is_acceptable=True)


Original Output: Artificial intelligence is the simulation of human intelligence in machines.
Sanitized Output: Artificial intelligence is the simulation of human intelligence in machines.
Original Output: I hate the concept of AI, it's promoting violence.
Sanitized Output: I hate the concept of AI, it&#x27;s promoting violence.
Original Output: Hello Hello Hello
Sanitized Output: Hello Hello Hello
Original Output: <script>alert('malicious');</script>
Sanitized Output: &lt;script&gt;alert(&#x27;malicious&#x27;);&lt;/script&gt;
Original Output: Artificial intelligence is a field that <b>abuses</b> resources.
Sanitized Output: Artificial intelligence is a field that &lt;b&gt;abuses&lt;/b&gt; resources.


In [6]:
import re
import html
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import asyncio
import nest_asyncio

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

class OutputModeration:
    def __init__(self, toxic_keywords=None, repetition_threshold=3, max_output_length=500):
        self.toxic_keywords = toxic_keywords or {"hate", "violence", "malicious", "abuse", "harm", "terror"}
        self.repetition_threshold = repetition_threshold
        self.max_output_length = max_output_length
        self.previous_outputs = []
        self.scaler = StandardScaler()

    def hallucination_prevention(self, output: str, context: str) -> bool:
        """
        Check if the output aligns with the provided context to prevent hallucination.
        A hallucination occurs when the output is irrelevant or deviates from the context.
        """
        return context.lower() in output.lower()

    def treat_as_untrusted(self, output: str) -> str:
        """
        Treat output as untrusted and sanitize it for safe use.
        This includes escaping potentially dangerous characters.
        """
        return html.escape(output)

    def filter_toxicity(self, output: str) -> bool:
        """
        Detect and filter toxic content in the output using a simple keyword-based approach.
        """
        for word in self.toxic_keywords:
            if re.search(rf"\b{word}\b", output, re.IGNORECASE):
                return False  # Toxic content detected
        return True  # No toxicity detected

    def anomaly_detection(self, output: str) -> bool:
        """
        Detect anomalies in output using clustering (DBSCAN).
        This can detect outliers and flag unusual or unexpected outputs.
        """
        words = output.split()
        unique_words = set(words)

        # Check for excessive repetition
        if len(unique_words) < self.repetition_threshold:
            return False  # Repetition detected
        
        # Check for excessive length
        if len(output) > self.max_output_length:
            return False  # Output too long
        
        # Prepare data for clustering (for simplicity, use word counts)
        self.previous_outputs.append(output)
        word_counts = [len(output.split()) for output in self.previous_outputs]

        # Standardize and cluster using DBSCAN
        word_counts_scaled = self.scaler.fit_transform([[count] for count in word_counts])
        dbscan = DBSCAN(eps=0.5, min_samples=2)
        cluster_labels = dbscan.fit_predict(word_counts_scaled)
        
        # Anomaly if not belonging to a large cluster
        if cluster_labels[-1] == -1:  # Outlier
            return False  # Anomaly detected

        return True  # No anomalies detected

    def monitor_and_log(self, output: str, context: str, status: str):
        """
        Monitor output and log for analysis and debugging, including timestamps, context, and status.
        Logs are saved in a log file for future reference and analysis.
        """
        try:
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            with open("output_logs.txt", "a") as log_file:
                log_file.write(
                    f"[{timestamp}] Context: {context}\n"
                    f"Output: {output}\n"
                    f"Status: {status}\n\n"
                )
        except Exception as e:
            print(f"Error logging output: {e}")

    def feedback_mechanism(self, output: str, is_acceptable: bool):
        """
        Allow user feedback to refine future moderation decisions.
        This feedback helps in training and fine-tuning moderation rules over time.
        """
        feedback_file = "moderation_feedback.txt"
        try:
            with open(feedback_file, "a") as file:
                feedback = "Acceptable" if is_acceptable else "Unacceptable"
                file.write(f"Output: {output}\nFeedback: {feedback}\n\n")
        except Exception as e:
            print(f"Error saving feedback: {e}")


class AsyncOutputProcessor:
    def __init__(self, moderation_system: OutputModeration):
        self.moderation_system = moderation_system

    async def process_output_async(self, output: str, context: str):
        """
        Asynchronously process output for moderation.
        """
        # Treat output as untrusted and sanitize it
        safe_output = self.moderation_system.treat_as_untrusted(output)
        print(f"Sanitized Output: {safe_output}")

        # Check for hallucinations
        if not self.moderation_system.hallucination_prevention(safe_output, context):
            status = "Hallucinated or irrelevant content detected."
            print(f"Warning: {status}")
            self.moderation_system.monitor_and_log(safe_output, context, status)
            return

        # Filter for toxicity
        if not self.moderation_system.filter_toxicity(safe_output):
            status = "Toxic or harmful content detected."
            print(f"Warning: {status}")
            self.moderation_system.monitor_and_log(safe_output, context, status)
            self.moderation_system.feedback_mechanism(safe_output, False)
            return

        # Detect anomalies
        if not self.moderation_system.anomaly_detection(safe_output):
            status = "Anomaly detected in content (e.g., repetition, length, nonsensical patterns)."
            print(f"Warning: {status}")
            self.moderation_system.monitor_and_log(safe_output, context, status)
            self.moderation_system.feedback_mechanism(safe_output, False)
            return

        # Log output for monitoring
        status = "Output accepted."
        self.moderation_system.monitor_and_log(safe_output, context, status)
        print("Output logged successfully.\n")

        # Collect user feedback (mock example for demonstration)
        self.moderation_system.feedback_mechanism(safe_output, is_acceptable=True)


# Main Execution with Example Usage
if __name__ == "__main__":
    # Initialize the moderation system and processor
    moderation_system = OutputModeration()
    async_processor = AsyncOutputProcessor(moderation_system)

    # Example context and outputs
    context = "Explain the concept of artificial intelligence."
    outputs = [
        "Artificial intelligence is the simulation of human intelligence in machines.",
        "I hate the concept of AI, it's promoting violence.",
        "Hello Hello Hello",
        "<script>alert('malicious');</script>",
        "Artificial intelligence is a field that <b>abuses</b> resources.",
        "AI can be a tool for both good and evil. However, there are concerns about misuse."
    ]

    # Use asynchronous processing for outputs
    loop = asyncio.get_event_loop()
    tasks = [async_processor.process_output_async(output, context) for output in outputs]
    loop.run_until_complete(asyncio.gather(*tasks))


Sanitized Output: Artificial intelligence is the simulation of human intelligence in machines.
Sanitized Output: I hate the concept of AI, it&#x27;s promoting violence.
Sanitized Output: Hello Hello Hello
Sanitized Output: &lt;script&gt;alert(&#x27;malicious&#x27;);&lt;/script&gt;
Sanitized Output: Artificial intelligence is a field that &lt;b&gt;abuses&lt;/b&gt; resources.
Sanitized Output: AI can be a tool for both good and evil. However, there are concerns about misuse.


In [ ]:
import re
import html
from datetime import datetime
import asyncio
import nest_asyncio
import nemo
import nemo.collections.nlp as nemo_nlp
from nemo.collections.nlp.models import TextClassificationModel
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN

# Apply nest_asyncio to allow nested event loops
nest_asyncio.apply()

class OutputModeration:
    def __init__(self, toxic_keywords=None, repetition_threshold=3, max_output_length=500):
        self.toxic_keywords = toxic_keywords or {"hate", "violence", "malicious", "abuse", "harm", "terror"}
        self.repetition_threshold = repetition_threshold
        self.max_output_length = max_output_length
        self.previous_outputs = []
        self.scaler = StandardScaler()

        # Load NeMo model for toxicity detection (this model is just an example)
        self.toxicity_model = nemo_nlp.models.TextClassificationModel.from_pretrained('nvidia/bert-base-uncased')
    
    def hallucination_prevention(self, output: str, context: str) -> bool:
        """
        Check if the output aligns with the provided context to prevent hallucination.
        """
        return context.lower() in output.lower()

    def treat_as_untrusted(self, output: str) -> str:
        """
        Treat output as untrusted and sanitize it for safe use.
        """
        return html.escape(output)

    def filter_toxicity(self, output: str) -> bool:
        """
        Detect and filter toxic content using NeMo model.
        """
        # Use NeMo model for toxicity detection (output is tokenized and predicted)
        predictions = self.toxicity_model.predict([output])
        # Assuming model predicts 0 = non-toxic, 1 = toxic
        if predictions[0] == 1:  # Toxic content detected
            return False
        return True  # No toxicity detected

    def anomaly_detection(self, output: str) -> bool:
        """
        Detect anomalies in output using clustering (DBSCAN).
        """
        words = output.split()
        unique_words = set(words)

        if len(unique_words) < self.repetition_threshold:
            return False  # Repetition detected
        
        if len(output) > self.max_output_length:
            return False  # Output too long

        # Prepare data for clustering (for simplicity, use word counts)
        self.previous_outputs.append(output)
        word_counts = [len(output.split()) for output in self.previous_outputs]

        # Standardize and cluster using DBSCAN
        word_counts_scaled = self.scaler.fit_transform([[count] for count in word_counts])
        dbscan = DBSCAN(eps=0.5, min_samples=2)
        cluster_labels = dbscan.fit_predict(word_counts_scaled)
        
        # Anomaly if not belonging to a large cluster
        if cluster_labels[-1] == -1:  # Outlier
            return False  # Anomaly detected

        return True  # No anomalies detected

    def monitor_and_log(self, output: str, context: str, status: str):
        """
        Monitor output and log for analysis and debugging, including timestamps, context, and status.
        """
        try:
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            with open("output_logs.txt", "a") as log_file:
                log_file.write(
                    f"[{timestamp}] Context: {context}\n"
                    f"Output: {output}\n"
                    f"Status: {status}\n\n"
                )
        except Exception as e:
            print(f"Error logging output: {e}")

    def feedback_mechanism(self, output: str, is_acceptable: bool):
        """
        Allow user feedback to refine future moderation decisions.
        """
        feedback_file = "moderation_feedback.txt"
        try:
            with open(feedback_file, "a") as file:
                feedback = "Acceptable" if is_acceptable else "Unacceptable"
                file.write(f"Output: {output}\nFeedback: {feedback}\n\n")
        except Exception as e:
            print(f"Error saving feedback: {e}")


class AsyncOutputProcessor:
    def __init__(self, moderation_system: OutputModeration):
        self.moderation_system = moderation_system

    async def process_output_async(self, output: str, context: str):
        """
        Asynchronously process output for moderation.
        """
        # Treat output as untrusted and sanitize it
        safe_output = self.moderation_system.treat_as_untrusted(output)
        print(f"Sanitized Output: {safe_output}")

        # Check for hallucinations
        if not self.moderation_system.hallucination_prevention(safe_output, context):
            status = "Hallucinated or irrelevant content detected."
            print(f"Warning: {status}")
            self.moderation_system.monitor_and_log(safe_output, context, status)
            return

        # Filter for toxicity using NeMo model
        if not self.moderation_system.filter_toxicity(safe_output):
            status = "Toxic or harmful content detected."
            print(f"Warning: {status}")
            self.moderation_system.monitor_and_log(safe_output, context, status)
            self.moderation_system.feedback_mechanism(safe_output, False)
            return

        # Detect anomalies
        if not self.moderation_system.anomaly_detection(safe_output):
            status = "Anomaly detected in content (e.g., repetition, length, nonsensical patterns)."
            print(f"Warning: {status}")
            self.moderation_system.monitor_and_log(safe_output, context, status)
            self.moderation_system.feedback_mechanism(safe_output, False)
            return

        # Log output for monitoring
        status = "Output accepted."
        self.moderation_system.monitor_and_log(safe_output, context, status)
        print("Output logged successfully.\n")

        # Collect user feedback (mock example for demonstration)
        self.moderation_system.feedback_mechanism(safe_output, is_acceptable=True)


# Main Execution with Example Usage
if __name__ == "__main__":
    # Initialize the moderation system and processor
    moderation_system = OutputModeration()
    async_processor = AsyncOutputProcessor(moderation_system)

    # Example context and outputs
    context = "Explain the concept of artificial intelligence."
    outputs = [
        "Artificial intelligence is the simulation of human intelligence in machines.",
        "I hate the concept of AI, it's promoting violence.",
        "Hello Hello Hello",
        "<script>alert('malicious');</script>",
        "Artificial intelligence is a field that <b>abuses</b> resources.",
        "AI can be a tool for both good and evil. However, there are concerns about misuse."
    ]

    # Use asynchronous processing for outputs
    loop = asyncio.get_event_loop()
    tasks = [async_processor.process_output_async(output, context) for output in outputs]
    loop.run_until_complete(asyncio.gather(*tasks))


# Conclusion

This notebook provides a **foundational framework** for implementing output moderation in AI systems. The techniques demonstrated here—such as hallucination prevention, toxicity filtering, anomaly detection, and output sanitization—highlight the critical steps needed to ensure that AI-generated content is safe, relevant, and aligned with user expectations.

It is important to note that this code serves as a **basic example**, emphasizing high-level principles rather than exhaustive implementation. Developers can use this as a starting point and expand upon it to address specific use cases, incorporate advanced moderation techniques, and integrate with production-level systems. 

Effective moderation is essential for building trust in AI applications, especially in sensitive or high-stakes domains. This notebook is a reminder of the key considerations that should guide the design of robust and ethical AI systems. Further refinements and domain-specific adjustments are encouraged to fully realize the potential of these principles in practice.
